# Leaflet

Leaflet es una librería de código abierto desarrollada en _JavaScript_ para mapas interactivos, la librería `ipyleaflet` es un _wrapper_ para Python.

Carga de librerías

In [1]:
import os
import yaml
os.environ['USE_PYGEOS'] = '0'

import pandas as pd
import geopandas as gpd

from geopandas.tools import geocode
from ipyleaflet import Map, basemaps, Marker, basemap_to_tiles

Ruta relativa a archivos de datos y carga de credenciales de API's

In [2]:
# Rutas de archivo
ROOT = os.path.dirname(os.getcwd())
data_path = os.path.join(ROOT,'data','Data pt1')

# Credenciales de api
with open(os.path.join(ROOT, 'credentials.yml'), 'r') as file:
    credentials = yaml.safe_load(file)

Nuevamente se utilizarán datos que no están georeferenciados y que, en cambio, cuentan con una dirección de las ubicaciones de interés. Por esta razón, es necesario hacer un _geocoding_ de las direcciones.

**Paso 1**: Carga de datos originales

In [3]:
dc_attractions = pd.read_csv(os.path.join(data_path,'DC_Attractions.csv'))
dc_attractions

,Attraction,Address
0,Washington Monument,"2 15th St NW, Washington, DC 20024"
1,Smithsonian National Air and Space Museum,"600 Independence Ave SW, Washington, DC 20560"
2,White House,"1600 Pennsylvania Avenue NW, Washington, DC 20500"


**Paso 2**: _Geocoding_ usando la API de **MapQuest**

In [4]:
# Georeferenciación de direcciones 
dc_attractions_gpd = geocode(
    strings=dc_attractions.Address
    ,provider='mapquest'
    ,timeout=4
    ,api_key=credentials['map_quest'])

# Unión de dataframes
dc_attractions_gpd = dc_attractions_gpd.join(dc_attractions[['Attraction']])

dc_attractions_gpd

,geometry,address,Attraction
0,POINT (-77.03296 38.88898),"2 15th St NW, Washington Mall, Washington, Dis...",Washington Monument
1,POINT (-77.02007 38.88756),"600 Independence Ave SW, Southwest Employment ...",Smithsonian National Air and Space Museum
2,POINT (-77.03655 38.89795),"1600 Pennsylvania Ave NW, Washington Mall, Was...",White House


**Paso 3**: Extracción de coordenadas como variables independientes

In [5]:
dc_attractions_gpd = (
    dc_attractions_gpd
    .assign(
        lon = lambda _df: _df.geometry.x
        ,lat = lambda _df: _df.geometry.y)
)

dc_attractions_gpd

,geometry,address,Attraction,lon,lat
0,POINT (-77.03296 38.88898),"2 15th St NW, Washington Mall, Washington, Dis...",Washington Monument,-77.03296,38.88898
1,POINT (-77.02007 38.88756),"600 Independence Ave SW, Southwest Employment ...",Smithsonian National Air and Space Museum,-77.02007,38.88756
2,POINT (-77.03655 38.89795),"1600 Pennsylvania Ave NW, Washington Mall, Was...",White House,-77.03655,38.89795


Para definir un mapa con _Leaflet_ es necesario crea una instancia de mapa indicando el **tipo de mapa**, el **punto de referencia** en torno al cual estará centrado el mapa, y el nivel de **zoom** que tendrá por defecto la visualización.

Para mayor detalle del tipo de mapa que se puede utilizar en las visualizaciones se puede consultar la [documentación](https://ipyleaflet.readthedocs.io/en/latest/map_and_basemaps/basemaps.html) de _Leaflet_

In [6]:
# Iniciar instancia de mapa
map = Map(
    basemap=basemap_to_tiles(basemap=basemaps.CartoDB.Positron), # .Stamen.Toner
    center=(38.89951498583087, -77.03599825749647),
    zoom=12)

map

Map(center=[38.89951498583087, -77.03599825749647], controls=(ZoomControl(options=['position', 'zoom_in_text',…

Una vez creada la base, se van agregando los diferentes objetos a modo de capas en el mapa.

In [7]:
# Mapeo de coordenadas como marcadores
for index, row in dc_attractions_gpd.iterrows():
    # Crear marcador
    marker = Marker(
        location=[row.loc['lat'], row.loc['lon']]
        ,title=row.loc['Attraction'])

    # Agregar marcador al mapa
    map.add_layer(layer=marker)

map

Map(center=[38.89951498583087, -77.03599825749647], controls=(ZoomControl(options=['position', 'zoom_in_text',…

---